In [ ]:
import pandas as pd


/tmp/ipykernel_318041/1942004796.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data =  pd.read_csv("database/call_connected_issues.csv")


In [3]:
import pandas as pd

data = pd.read_csv("database/counselling_data.csv")
data.columns = data.columns.str.replace(" ", "_")

data.columns, data.shape




FileNotFoundError: [Errno 2] No such file or directory: 'database/counselling_data.csv'

In [9]:
data.to_csv("databse_preperation_csv.csv", index=False)

In [1]:
import os

In [2]:
os.listdir('database')

['info.txt',
 'Anonymized_Call_Handle_Data.csv',
 'repeated_calls.csv',
 'India_State_Boundary.prj',
 'India_State_Boundary.sbn',
 'INDIA_STATES.geojson',
 'Call_not_picked.csv',
 'India_State_Boundary.sbx',
 'call_connected_issues.csv',
 'counselling_data.csv',
 'India_State_Boundary.cpg',
 'India_State_Boundary.shx',
 'Telemana_Analysis.csv',
 'India_State_Boundary.dbf',
 'counselling_complaints.csv',
 'India_State_Boundary.shp.xml',
 'India_State_Boundary.shp']

In [3]:
import pandas as pd
import geopandas as gpd
import json
import os
from typing import List, Dict, Any

class MapDataBuilder:
    def __init__(self,
                 survey_csv: str = "counselling_data.csv",
                 states_geojson: str = "INDIA_STATES.geojson",
                 output_json: str = "static/q6_map_.json"):
        self.survey_csv  = survey_csv
        self.states_geojson = states_geojson
        self.output_json = output_json

    def load_and_prepare_survey(self) -> pd.DataFrame:
        df = pd.read_csv(self.survey_csv)
        df = df[["Patient_State", "patient â†’ age"]].copy()
        df.columns = ["Patient_State", "Patient_age"]

        # fill missing
        df["Patient_age"].fillna(df["Patient_age"].mean(), inplace=True)

        # categorize
        df["Age_Group"] = df["Patient_age"].apply(
            lambda age: "Children" if age <= 20
                        else "Adults" if age <= 60
                        else "Elders"
        )

        pct = (
            df.groupby(["Patient_State", "Age_Group"]).size()
              .unstack(fill_value=0)
              .pipe(lambda d: d.div(d.sum(axis=1), axis=0) * 100)
              .reset_index()
        )
        for col in ["Children", "Adults", "Elders"]:
            if col not in pct:
                pct[col] = 0.0
        return pct

    def load_states_geo(self) -> gpd.GeoDataFrame:
        gdf = gpd.read_file(self.states_geojson)
        gdf["State_Name"] = gdf["ST_NM"].str.upper().str.strip()
        return gdf[["State_Name", "geometry"]]

    def build(self) -> List[Dict[str, Any]]:
        survey_pct = self.load_and_prepare_survey()
        gdf_states = self.load_states_geo()

        merged = pd.merge(
            survey_pct,
            gdf_states,
            left_on="Patient_State",
            right_on="State_Name",
            how="inner"
        )

        records: List[Dict[str, Any]] = []
        for _, row in merged.iterrows():
            records.append({
                "state": row["State_Name"],
                "geometry": row["geometry"].wkt,
                "Children": round(row["Children"], 6),
                "Adults":   round(row["Adults"],   6),
                "Elders":   round(row["Elders"],   6)
            })
        return records

    def save(self):
        os.makedirs(os.path.dirname(self.output_json), exist_ok=True)
        data = self.build()
        with open(self.output_json, "w") as f:
            json.dump(data, f, indent=2)
        print(f"✅ Saved map data to {self.output_json}")

    def run(self):
        self.save()


if __name__ == "__main__":
    builder = MapDataBuilder(
        survey_csv="database/counselling_data.csv",
        states_geojson="database/INDIA_STATES.geojson",
        output_json="static/q6_map_.json"
    )
    builder.run()


/tmp/ipykernel_306181/244843486.py:17: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.survey_csv)
/tmp/ipykernel_306181/244843486.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Patient_age"].fillna(df["Patient_age"].mean(), inplace=True)


KeyError: 'ST_NM'

In [4]:
os.listdir('database')

['info.txt',
 'Anonymized_Call_Handle_Data.csv',
 'repeated_calls.csv',
 'India_State_Boundary.prj',
 'India_State_Boundary.sbn',
 'INDIA_STATES.geojson',
 'Call_not_picked.csv',
 'India_State_Boundary.sbx',
 'call_connected_issues.csv',
 'counselling_data.csv',
 'India_State_Boundary.cpg',
 'India_State_Boundary.shx',
 'Telemana_Analysis.csv',
 'India_State_Boundary.dbf',
 'counselling_complaints.csv',
 'India_State_Boundary.shp.xml',
 'India_State_Boundary.shp']

In [6]:
import pandas as pd
data = pd.read_csv('database/counselling_complaints.csv')
data.shape

(999, 21)

In [9]:
import pandas as pd
import geopandas as gpd
import json
import os
from typing import List, Dict, Any

class MapDataBuilder:
    def __init__(self,
                 survey_csv: str = "counselling_data.csv",
                 states_geojson: str = "INDIA_STATES.geojson",
                 output_json: str = "static/q6_map_.json"):
        self.survey_csv = survey_csv
        self.states_geojson = states_geojson
        self.output_json = output_json

    def load_and_prepare_survey(self) -> pd.DataFrame:
        df = pd.read_csv(self.survey_csv)
        df = df[["Patient_State", "patient â†’ age"]].copy()
        df.columns = ["Patient_State", "Patient_age"]

        # Fill missing age values with mean
        df["Patient_age"].fillna(df["Patient_age"].mean(), inplace=True)

        # Categorize by age group
        df["Age_Group"] = df["Patient_age"].apply(
            lambda age: "Children" if age <= 20
                        else "Adults" if age <= 60
                        else "Elders"
        )

        # Calculate percentage per age group per state
        pct = (
            df.groupby(["Patient_State", "Age_Group"]).size()
              .unstack(fill_value=0)
              .pipe(lambda d: d.div(d.sum(axis=1), axis=0) * 100)
              .reset_index()
        )

        # Ensure all expected columns are present
        for col in ["Children", "Adults", "Elders"]:
            if col not in pct:
                pct[col] = 0.0

        return pct

    def load_states_geo(self) -> gpd.GeoDataFrame:
        gdf = gpd.read_file(self.states_geojson)

        # Print columns to help debug if something goes wrong
        print("🧾 Available columns in GeoJSON:", gdf.columns.tolist())

        # Try known column names for state
        for possible_col in ["ST_NM", "state", "STATE_NAME", "NAME_1", "NAME"]:
            if possible_col in gdf.columns:
                gdf["State_Name"] = gdf[possible_col].str.upper().str.strip()
                break
        else:
            raise ValueError("❌ Could not find a suitable state name column in GeoJSON file.")

        return gdf[["State_Name", "geometry"]]

    def build(self) -> List[Dict[str, Any]]:
        survey_pct = self.load_and_prepare_survey()
        gdf_states = self.load_states_geo()

        # Merge survey data with geographic shapes
        merged = pd.merge(
            survey_pct,
            gdf_states,
            left_on="Patient_State",
            right_on="State_Name",
            how="inner"
        )

        # Prepare JSON records
        records: List[Dict[str, Any]] = []
        for _, row in merged.iterrows():
            records.append({
                "state": row["State_Name"],
                "geometry": row["geometry"].wkt,
                "Children": round(row["Children"], 6),
                "Adults": round(row["Adults"], 6),
                "Elders": round(row["Elders"], 6)
            })
        return records

    def save(self):
        os.makedirs(os.path.dirname(self.output_json), exist_ok=True)
        data = self.build()
        with open(self.output_json, "w") as f:
            json.dump(data, f, indent=2)
        print(f"✅ Saved map data to {self.output_json}")

    def run(self):
        self.save()


if __name__ == "__main__":
    builder = MapDataBuilder(
        survey_csv="database/counselling_data.csv",
        states_geojson="database/INDIA_STATES.geojson",
        output_json="static/q6_map_.json"
    )
    builder.run()


/tmp/ipykernel_306181/14657156.py:17: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.survey_csv)
/tmp/ipykernel_306181/14657156.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Patient_age"].fillna(df["Patient_age"].mean(), inplace=True)


🧾 Available columns in GeoJSON: ['STNAME', 'STCODE11', 'STNAME_SH', 'Shape_Length', 'Shape_Area', 'OBJECTID_1', 'OBJECTID', 'State_LGD', 'Shape_Leng', 'MaxSimpTol', 'MinSimpTol', 'geometry']


ValueError: ❌ Could not find a suitable state name column in GeoJSON file.